# 0. Setup

Import packages and specify any important functions here.

In [1]:
# import standard python packages
import pandas as pd
import subprocess
import os
import dill

# add the utils and env directories to the path
import sys
sys.path.append('../../utils/')
sys.path.append('../../env/')

# import functions from utils directory files
from string_functions import *
from biofile_handling import *

# import paths to software installs from env
from install_locs import *

# 1. Download and describe data

This notebook collects data from the [Liao et al. 2022](https://www.sciencedirect.com/science/article/pii/S0092867418301168#sec4) Xenopus laevis adult cell atlas.

## Dataset description

- Data can be found at the [GEO Accession GSE195790](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE195790).
- This notebook previously collects data from one sample, ["Xenopus_brain_COL65"](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM6214268), for the purposes of data analysis and exploration.

In [2]:
################
# general info #
################

# Specify the name of the species folder in Amazon S3
species = 'Xenopus_laevis'

# Specify any particular identifying conditions, eg tissue type:
conditions = 'adultbrain'

# Specify url and other variables
genome_fasta_url = 'https://ftp.xenbase.org/pub/Genomics/JGI/Xenla9.2/XENLA_9.2_genome.fa.gz'
genome_version = 'XENLA_9.2'

annot_url = 'https://ftp.xenbase.org/pub/Genomics/JGI/Xenla9.2/XENLA_9.2_Xenbase.gtf'

### EXCEPTION ###
# Need to get a second annotation in order to pull usable ID mapping information
annot2_url = 'https://ftp.xenbase.org/pub/Genomics/JGI/Xenla9.2/XENLA_9.2_GCA.gff3'
#################

gxc_url = 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM6214nnn/GSM6214268/suppl/GSM6214268_Xenopus_brain_COL65_dge.txt.gz'

###########
# runtime #
###########

protocol = 'curl'

species_prefix = prefixify(species)

# Specify folder as destination for file downloads
output_folder = '../../output/' + prefixify(species) + '_' + conditions + '/'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
species_SampleDict = SampleDict(species, conditions, output_folder)

genome_fasta = GenomeFastaFile(
    filename = '',
    sampledict = species_SampleDict,
    version = genome_version,
    url = genome_fasta_url,
    protocol = protocol
)

annot = GenomeGtfFile(
    filename = '',
    sampledict = species_SampleDict,
    GenomeFastaFile = genome_fasta,
    url = annot_url,
    protocol = protocol
)

annot2 = GenomeGffFile(
    filename = '',
    sampledict = species_SampleDict,
    GenomeFastaFile = genome_fasta,
    url = annot2_url,
    protocol = protocol
)

gxc = GxcFile(
    filename = '',
    sampledict = species_SampleDict,
    GenomeFastaFile = genome_fasta,
    GenomeAnnotFile = annot,
    url = gxc_url,
    protocol = protocol
)


sample_Docket = Docket(species_SampleDict)
keyfiles = {
    'annot': annot,
    'annot2': annot2,
    'genome_fasta': genome_fasta,
    'gxc': gxc
}
sample_Docket.add_keyfiles(keyfiles)

display(vars(sample_Docket))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  730M  100  730M    0     0  19.3M      0  0:00:37  0:00:37 --:--:-- 16.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  109M  100  109M    0     0  6503k      0  0:00:17  0:00:17 --:--:-- 5801k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  150M  100  150M    0     0  26.4M      0  0:00:05  0:00:05 --:--:-- 30.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.7M  100 18.7M    0     0  8600k      0  0:00:02  0:00:02 --:--:-- 8605k


{'species': 'Xenopus_laevis',
 'conditions': 'adultbrain',
 'directory': '../../output/Xlae_adultbrain/',
 'files': {},
 'annot': <biofile_handling.GenomeGtfFile at 0x7f981479b8b0>,
 'annot2': <biofile_handling.GenomeGffFile at 0x7f981479b730>,
 'genome_fasta': <biofile_handling.GenomeFastaFile at 0x7f976386f2e0>,
 'gxc': <biofile_handling.GxcFile at 0x7f981479b940>}

# 2. Load in the gxc matrix and get gene names

In [3]:
genes_matrix = pd.read_csv(sample_Docket.gxc.path, sep = '\t')
display(genes_matrix)

gxc_genes_list = pd.DataFrame({'gene_name':genes_matrix['GENE']})
display(gxc_genes_list)

,GENE,AACCTATTCATATAAGGG,CTCGCATCAAAGTTAACT,AACCTAGTATACTTCCGC,AACCTAAAAGTTCTGAAA,CTCGCACGCACCCTCCAT,ACGTTGTATTGTAGCGAG,ACGAGCATGCTTTAGTCG,AACCTAGTCCCGCCATCT,AACCTAGCGAATTAGAGA,...,TCACTTGTTGCCATGCTT,TCGGGTTGTCACACTTAT,TCGTAATCGTAAGTTGCC,TGTCACGAATTACACAAG,TGTGCGTACTTCTAGTCG,TTAACTATACAGTGGATG,TTGGACACTTATGATCTT,AAAGTTACTTATGCCCTC,AACCTACGCACCTGCGGA,AACCTATAGTCGCTGTGT
0,3.S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,42Sp43.L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42Sp50.L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AK6.L,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AK6.S,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25907,zyg11b.L,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25908,zyg11b.S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25909,zzef1.S,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25910,zzz3.L,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


,gene_name
0,3.S
1,42Sp43.L
2,42Sp50.L
3,AK6.L
4,AK6.S
...,...
25907,zyg11b.L
25908,zyg11b.S
25909,zzef1.S
25910,zzz3.L


# 3. Get mapping identifiers

In [37]:
# load in the original GFF-based annotation
models = pd.read_csv(sample_Docket.annot.path, skiprows = 0, header = None, sep = '\t', on_bad_lines = 'skip', comment='#')
display(models)

attributes_column = 8

# Check the structure of fields in the GFF additional fields section
display(models[attributes_column][0])

# Remove any rows with NaNs
models.dropna(inplace = True)

,0,1,2,3,4,5,6,7,8
0,MT,Xenbase,exon,2136,2204,.,+,.,"gene_id ""gene42065""; gene_name ""mt-trna-phe.L""..."
1,MT,Xenbase,exon,2205,3023,.,+,.,"gene_id ""gene34778""; gene_name ""mt-rnr1.L""; tr..."
2,MT,Xenbase,exon,3024,3092,.,+,.,"gene_id ""gene48202""; gene_name ""mt-trna-val.L""..."
3,MT,Xenbase,exon,3093,4723,.,+,.,"gene_id ""gene44770""; gene_name ""mt-rnr2.L""; tr..."
4,MT,Xenbase,exon,4724,4798,.,+,.,"gene_id ""gene43253""; gene_name ""mt-trna-leu1.L..."
...,...,...,...,...,...,...,...,...,...
771995,chr9_10S,Xenbase,CDS,104527008,104527068,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771996,chr9_10S,Xenbase,exon,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771997,chr9_10S,Xenbase,CDS,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771998,chr9_10S,Xenbase,exon,104531027,104531054,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."


'gene_id "gene42065"; gene_name "mt-trna-phe.L"; transcript_id "gene42065_t"; transcript_name "mt-trna-phe.L";'

In [38]:
# Extract field and database cross-ref (dbxref) information into columns
models['field_dictionary'] = models[attributes_column].apply(convert_fields_to_dict_gtf)
models['gene_name'] = [d.get('gene_name') for d in models['field_dictionary']]

display(models)
display(len(models['gene_name'].unique()))

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name
0,MT,Xenbase,exon,2136,2204,.,+,.,"gene_id ""gene42065""; gene_name ""mt-trna-phe.L""...","{'gene_id': 'gene42065', 'gene_name': 'mt-trna...",mt-trna-phe.L
1,MT,Xenbase,exon,2205,3023,.,+,.,"gene_id ""gene34778""; gene_name ""mt-rnr1.L""; tr...","{'gene_id': 'gene34778', 'gene_name': 'mt-rnr1...",mt-rnr1.L
2,MT,Xenbase,exon,3024,3092,.,+,.,"gene_id ""gene48202""; gene_name ""mt-trna-val.L""...","{'gene_id': 'gene48202', 'gene_name': 'mt-trna...",mt-trna-val.L
3,MT,Xenbase,exon,3093,4723,.,+,.,"gene_id ""gene44770""; gene_name ""mt-rnr2.L""; tr...","{'gene_id': 'gene44770', 'gene_name': 'mt-rnr2...",mt-rnr2.L
4,MT,Xenbase,exon,4724,4798,.,+,.,"gene_id ""gene43253""; gene_name ""mt-trna-leu1.L...","{'gene_id': 'gene43253', 'gene_name': 'mt-trna...",mt-trna-leu1.L
...,...,...,...,...,...,...,...,...,...,...,...
771995,chr9_10S,Xenbase,CDS,104527008,104527068,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S
771996,chr9_10S,Xenbase,exon,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S
771997,chr9_10S,Xenbase,CDS,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S
771998,chr9_10S,Xenbase,exon,104531027,104531054,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S


49282

In [39]:
models2 = pd.read_csv(sample_Docket.annot2.path, skiprows = 0, header = None, sep = '\t', on_bad_lines = 'skip', comment='#')
display(models2)

,0,1,2,3,4,5,6,7,8
0,chr1L,Genbank,gene,17924,18399,.,-,.,"ID=gene0;Name=Xelaev18004747m;end_range=18399,..."
1,chr1L,Genbank,mRNA,17924,18399,.,-,.,ID=rna0;Parent=gene0;Note=transcript XELAEV_18...
2,chr1L,Genbank,exon,18336,18399,.,-,.,ID=id1;Parent=rna0;Note=transcript XELAEV_1800...
3,chr1L,Genbank,exon,17924,18243,.,-,.,ID=id2;Parent=rna0;Note=transcript XELAEV_1800...
4,chr1L,Genbank,CDS,18336,18399,.,-,0,ID=cds0;Parent=rna0;Dbxref=Phytozome:Xelaev180...
...,...,...,...,...,...,...,...,...,...
809901,Scaffold94051,Genbank,CDS,8,205,.,+,0,ID=cds47730;Parent=rna47730;Dbxref=Phytozome:X...
809902,Scaffold95291,Genbank,gene,9,236,.,-,.,ID=gene45941;Name=Xelaev18004691m;end_range=23...
809903,Scaffold95291,Genbank,mRNA,9,236,.,-,.,ID=rna47731;Parent=gene45941;Note=transcript X...
809904,Scaffold95291,Genbank,exon,9,236,.,-,.,ID=id452950;Parent=rna47731;Note=transcript XE...


In [40]:
models2['field_dictionary'] = models2[attributes_column].apply(convert_fields_to_dict_gff)
models2['gene_name'] = [d.get('Name') for d in models2['field_dictionary']]
models2['Xenbase'] = [d.get('Alias') for d in models2['field_dictionary']]
display(models2)

xenbase_keys = models2[['gene_name', 'Xenbase']].dropna().drop_duplicates()
xenbase_keys = xenbase_keys.groupby('gene_name').agg({'gene_name': 'first', 'Xenbase': ','.join}).reset_index(drop = True)
display(xenbase_keys)

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name,Xenbase
0,chr1L,Genbank,gene,17924,18399,.,-,.,"ID=gene0;Name=Xelaev18004747m;end_range=18399,...","{'ID': 'gene0', 'Name': 'Xelaev18004747m', 'en...",Xelaev18004747m,XB-GENE-5942444
1,chr1L,Genbank,mRNA,17924,18399,.,-,.,ID=rna0;Parent=gene0;Note=transcript XELAEV_18...,"{'ID': 'rna0', 'Parent': 'gene0', 'Note': 'tra...",None,None
2,chr1L,Genbank,exon,18336,18399,.,-,.,ID=id1;Parent=rna0;Note=transcript XELAEV_1800...,"{'ID': 'id1', 'Parent': 'rna0', 'Note': 'trans...",None,None
3,chr1L,Genbank,exon,17924,18243,.,-,.,ID=id2;Parent=rna0;Note=transcript XELAEV_1800...,"{'ID': 'id2', 'Parent': 'rna0', 'Note': 'trans...",None,None
4,chr1L,Genbank,CDS,18336,18399,.,-,0,ID=cds0;Parent=rna0;Dbxref=Phytozome:Xelaev180...,"{'ID': 'cds0', 'Parent': 'rna0', 'Dbxref': 'Ph...",OCT98948.1,None
...,...,...,...,...,...,...,...,...,...,...,...,...
809901,Scaffold94051,Genbank,CDS,8,205,.,+,0,ID=cds47730;Parent=rna47730;Dbxref=Phytozome:X...,"{'ID': 'cds47730', 'Parent': 'rna47730', 'Dbxr...",OCT55143.1,None
809902,Scaffold95291,Genbank,gene,9,236,.,-,.,ID=gene45941;Name=Xelaev18004691m;end_range=23...,"{'ID': 'gene45941', 'Name': 'Xelaev18004691m',...",Xelaev18004691m,None
809903,Scaffold95291,Genbank,mRNA,9,236,.,-,.,ID=rna47731;Parent=gene45941;Note=transcript X...,"{'ID': 'rna47731', 'Parent': 'gene45941', 'Not...",None,None
809904,Scaffold95291,Genbank,exon,9,236,.,-,.,ID=id452950;Parent=rna47731;Note=transcript XE...,"{'ID': 'id452950', 'Parent': 'rna47731', 'Note...",None,None


,gene_name,Xenbase
0,42sp43.L,XB-GENE-6252610
1,42sp50.L,XB-GENE-5853356
2,LOC100036702.L,XB-GENE-18005853
3,LOC100037907.L,XB-GENE-18005786
4,LOC100037907.S,XB-GENE-18005873
...,...,...
21951,zyx.L,XB-GENE-6252950
21952,zyx.S,XB-GENE-6253946
21953,zzef1.S,XB-GENE-6488145
21954,zzz3.L,XB-GENE-6486634


In [41]:
models = models.merge(xenbase_keys, on = 'gene_name', how = 'left')
new_models = models.copy(deep = True)
display(len(new_models[['gene_name', 'Xenbase']].dropna().drop_duplicates()))

new_models['Xenbase'] = new_models['Xenbase'].astype(str)
new_models.apply(lambda x: x['field_dictionary'].update({'Xenbase': x['Xenbase']}), axis = 1)
new_models['new_fields'] = new_models['field_dictionary'].apply(convert_dict_to_fields_gtf)

new_models = new_models[[0, 1, 2, 3, 4, 5, 6, 7, 'new_fields']]

models_asgtf_updated_filename = sample_Docket.annot.filename.replace('.gtf', '_updated.gtf')
models_asgtf_updated = GenomeGtfFile(models_asgtf_updated_filename, species_SampleDict, GenomeFastaFile = sample_Docket.genome_fasta)

new_models.to_csv(models_asgtf_updated.path, sep = '\t', index = None, header = None)

display(new_models)

19286

,0,1,2,3,4,5,6,7,new_fields
0,MT,Xenbase,exon,2136,2204,.,+,.,"gene_id ""gene42065""; gene_name ""mt-trna-phe.L""..."
1,MT,Xenbase,exon,2205,3023,.,+,.,"gene_id ""gene34778""; gene_name ""mt-rnr1.L""; tr..."
2,MT,Xenbase,exon,3024,3092,.,+,.,"gene_id ""gene48202""; gene_name ""mt-trna-val.L""..."
3,MT,Xenbase,exon,3093,4723,.,+,.,"gene_id ""gene44770""; gene_name ""mt-rnr2.L""; tr..."
4,MT,Xenbase,exon,4724,4798,.,+,.,"gene_id ""gene43253""; gene_name ""mt-trna-leu1.L..."
...,...,...,...,...,...,...,...,...,...
771995,chr9_10S,Xenbase,CDS,104527008,104527068,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771996,chr9_10S,Xenbase,exon,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771997,chr9_10S,Xenbase,CDS,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771998,chr9_10S,Xenbase,exon,104531027,104531054,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."


# 4. Extract gene IDs for mapping to UniprotKB
Specify which set of identifiers will be use to query the [Uniprot ID Mapping Tool](https://www.uniprot.org/id-mapping) via API.

If using an identifier from the `dbxref_dict`, specify the name via string in the `dbxref_datafield` variable.

In [42]:
dbxref_datafield = ''
datafield = 'Xenbase'

if dbxref_datafield != '':
    models.dropna(axis = 0, subset = ['dbxref_dict'], inplace = True)
    models[dbxref_datafield] = [d.get(dbxref_datafield) for d in models['dbxref_dict']]

    models_subset = models[['gene_name', dbxref_datafield]].dropna().drop_duplicates()
    display(models_subset)

elif datafield == 'gene_name':
    models_subset = models[['gene_name']].dropna().drop_duplicates()
    display(models_subset)

elif datafield != '':
    models_subset = models[['gene_name', datafield]].dropna().drop_duplicates()
    display(models_subset)

else:
    raise Exception('You must provide a data field for ID mapping.')

,gene_name,Xenbase
192,vdac2.S,XB-GENE-954844
212,samd8.S,XB-GENE-17343426
227,dusp13.S,XB-GENE-17337763
258,adk.S,XB-GENE-997231
281,vcl.S,XB-GENE-5759162
...,...,...
771494,nif3l1.S,XB-GENE-6078750
771661,tpsg1.S,XB-GENE-5893017
771750,loc100491461.S,XB-GENE-6489054
771854,rnf112.1.S,XB-GENE-5960855


# 5. Generate gene list file to query Uniprot ID Mapping API
Generate a text file ending in `_ids.txt` for sending to the ID mapping API.

In [43]:
datafield = dbxref_datafield if dbxref_datafield != '' else datafield

gene_list = models_subset[datafield].unique()

genelist_object = GeneListFile(
    filename = '', # filename is automatically generated from sampledict
    sampledict = species_SampleDict,
    sources = [sample_Docket.annot],
    genes = gene_list,
    identifier = datafield
    )

Wrote 19285 gene ids to ../../output/Xlae_adultbrain/Xlae_adultbrain_Xenbase_ids.txt


# 6. Query Uniprot ID Mapping API
Specify the `from_type` variable based on the Uniprot name of the identifier.
The table below lists some databases and the `from_type` string that the API accepts for that datatype.

For some reason, the ID mapping API usually returns "resource not found" the first time you use it.
Try re-running this cell if you receive that error.


| datatype | `from_type` string | description |
| ---: | :--- | :--- |
| Mouse Genome Informatics | `MGI` | ID starts with `MGI:` |
| Zebrafish Information Network | `ZFIN` | ID starts with `ZDB-GENE-` |
| NCBI Gene ID | `GeneID` | varies, usually numeric string |

In [44]:
from_type = 'Xenbase'
to_type = 'UniProtKB'

uniprot_idmm_object = genelist_object.get_uniprot_ids(ID_MAPPER_LOC, from_type, to_type)
sample_Docket.add_keyfiles({'uniprot_idmm': uniprot_idmm_object})

uniprot_idmm = pd.read_csv(sample_Docket.uniprot_idmm.path, sep = '\t')
display(uniprot_idmm)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  305k    0    52  100  305k     28   168k  0:00:01  0:00:01 --:--:--  168k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    24    0    24    0     0     38      0 --:--:-- --:--:-- --:--:--    38


,From,Entry,Entry Name,Reviewed,Protein names,Gene Names,Organism,Length
0,XB-GENE-954844,Q52KY1,Q52KY1_XENLA,unreviewed,Voltage-dependent anion-selective channel prot...,vdac2.S vdac2 vdac2.L,Xenopus laevis (African clawed frog),283
1,XB-GENE-997231,Q6DJM1,Q6DJM1_XENLA,unreviewed,Adenosine kinase (AK) (EC 2.7.1.20) (Adenosine...,adk.S adk MGC82032,Xenopus laevis (African clawed frog),361
2,XB-GENE-5910051,Q6GM53,IKKA_XENLA,reviewed,Inhibitor of nuclear factor kappa-B kinase sub...,chuk ikka,Xenopus laevis (African clawed frog),743
3,XB-GENE-958583,Q641H0,Q641H0_XENLA,unreviewed,Annexin,anxa11.S anx11 anxa11 cap50 MGC81584,Xenopus laevis (African clawed frog),502
4,XB-GENE-6251658,Q5FWP7,Q5FWP7_XENLA,unreviewed,MGC84996 protein (nuclear receptor coactivator...,ncoa4.S ara70 ele1 MGC84996 ncoa4 ncoa4-b ptc3...,Xenopus laevis (African clawed frog),625
...,...,...,...,...,...,...,...,...
17505,XB-GENE-6078750,A0A1L8ERE3,A0A1L8ERE3_XENLA,unreviewed,NIF3-like protein 1,nif3l1.S nif3l1 nif3l1.L,Xenopus laevis (African clawed frog),369
17506,XB-GENE-6078750,Q0IHC9,Q0IHC9_XENLA,unreviewed,NIF3-like protein 1,nif3l1.S MGC154449 nif3l1 nif3l1.L,Xenopus laevis (African clawed frog),344
17507,XB-GENE-5893017,A0A1L8ERF3,A0A1L8ERF3_XENLA,unreviewed,serine protease 27,tpsg1.S,Xenopus laevis (African clawed frog),280
17508,XB-GENE-5960855,A0A1L8ERI6,A0A1L8ERI6_XENLA,unreviewed,RING finger protein 112,rnf112.1.S,Xenopus laevis (African clawed frog),625


# 7. Extract results and generate Uniprot IDMM
Generates an idmm that links `gene_name`, the `dbxref_datafield` seleted above, and `uniprot_id` returned by API.

In [45]:
uniprot_idpairs = uniprot_idmm[['From', 'Entry']]
uniprot_idpairs.rename(columns = {'From': datafield, 'Entry': 'uniprot_id'}, inplace = True)
uniprot_idpairs[datafield] = uniprot_idpairs[datafield].astype(str)
display(uniprot_idpairs)

uniprot_output_idmm = models_subset.merge(uniprot_idpairs, on = datafield)
display(uniprot_output_idmm)

/tmp/ipykernel_9203/899271637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniprot_idpairs.rename(columns = {'From': datafield, 'Entry': 'uniprot_id'}, inplace = True)
/tmp/ipykernel_9203/899271637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniprot_idpairs[datafield] = uniprot_idpairs[datafield].astype(str)


,Xenbase,uniprot_id
0,XB-GENE-954844,Q52KY1
1,XB-GENE-997231,Q6DJM1
2,XB-GENE-5910051,Q6GM53
3,XB-GENE-958583,Q641H0
4,XB-GENE-6251658,Q5FWP7
...,...,...
17505,XB-GENE-6078750,A0A1L8ERE3
17506,XB-GENE-6078750,Q0IHC9
17507,XB-GENE-5893017,A0A1L8ERF3
17508,XB-GENE-5960855,A0A1L8ERI6


,gene_name,Xenbase,uniprot_id
0,vdac2.S,XB-GENE-954844,Q52KY1
1,adk.S,XB-GENE-997231,Q6DJM1
2,chuk.S,XB-GENE-5910051,Q6GM53
3,anxa11.S,XB-GENE-958583,Q641H0
4,ncoa4.S,XB-GENE-6251658,Q5FWP7
...,...,...,...
17474,nif3l1.S,XB-GENE-6078750,A0A1L8ERE3
17475,nif3l1.S,XB-GENE-6078750,Q0IHC9
17476,tpsg1.S,XB-GENE-5893017,A0A1L8ERF3
17477,rnf112.1.S,XB-GENE-5960855,A0A1L8ERI6


In [46]:
# generate a filename and file for the idmm
uniprot_output_idmm_filename = '_'.join([species_prefix, conditions, 'uniprot-idmm.tsv'])
uniprot_output_idmm_object = IdmmFile(uniprot_output_idmm_filename, species_SampleDict, kind = 'uniprot_idmm', sources = [sample_Docket.annot])

# save to file and add to the Docket
uniprot_output_idmm.to_csv(uniprot_output_idmm_object.path, sep = '\t')
sample_Docket.add_keyfile(uniprot_output_idmm_object, 'uniprot_idmm')

# 9. Generate gtf-idmm
This file maps the `gene_name` to `gene_id` and `transcript_id` fields generated by the conversion from GFF to GTF, which will be needed for downstream processing.

In [47]:
models_asgtf_df = pd.read_csv(models_asgtf_updated.path, sep = '\t', header = None)

display(models_asgtf_df)
display(models_asgtf_df[attributes_column][1])

,0,1,2,3,4,5,6,7,8
0,MT,Xenbase,exon,2136,2204,.,+,.,"gene_id ""gene42065""; gene_name ""mt-trna-phe.L""..."
1,MT,Xenbase,exon,2205,3023,.,+,.,"gene_id ""gene34778""; gene_name ""mt-rnr1.L""; tr..."
2,MT,Xenbase,exon,3024,3092,.,+,.,"gene_id ""gene48202""; gene_name ""mt-trna-val.L""..."
3,MT,Xenbase,exon,3093,4723,.,+,.,"gene_id ""gene44770""; gene_name ""mt-rnr2.L""; tr..."
4,MT,Xenbase,exon,4724,4798,.,+,.,"gene_id ""gene43253""; gene_name ""mt-trna-leu1.L..."
...,...,...,...,...,...,...,...,...,...
771995,chr9_10S,Xenbase,CDS,104527008,104527068,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771996,chr9_10S,Xenbase,exon,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771997,chr9_10S,Xenbase,CDS,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."
771998,chr9_10S,Xenbase,exon,104531027,104531054,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942..."


'gene_id "gene34778"; gene_name "mt-rnr1.L"; transcript_id "gene34778_t"; transcript_name "mt-rnr1.L"; Xenbase "nan"'

In [48]:
# Use a custom function to extract useful fields from the additional fields section (column 8)
# Pull from that dict to fill in additional useful columns
models_asgtf_df['field_dictionary'] = models_asgtf_df[attributes_column].apply(convert_fields_to_dict_gtf)
models_asgtf_df['gene_name'] = [d.get('gene_name') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['gene_id'] = [d.get('gene_id') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['transcript_id'] = [d.get('transcript_id') for d in models_asgtf_df['field_dictionary']]

# Remove CDS annotations because they interfere with TransDecoder cDNA generation
models_asgtf_df = models_asgtf_df[models_asgtf_df[2] != 'CDS']
display(models_asgtf_df)

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name,gene_id,transcript_id
0,MT,Xenbase,exon,2136,2204,.,+,.,"gene_id ""gene42065""; gene_name ""mt-trna-phe.L""...","{'gene_id': 'gene42065', 'gene_name': 'mt-trna...",mt-trna-phe.L,gene42065,gene42065_t
1,MT,Xenbase,exon,2205,3023,.,+,.,"gene_id ""gene34778""; gene_name ""mt-rnr1.L""; tr...","{'gene_id': 'gene34778', 'gene_name': 'mt-rnr1...",mt-rnr1.L,gene34778,gene34778_t
2,MT,Xenbase,exon,3024,3092,.,+,.,"gene_id ""gene48202""; gene_name ""mt-trna-val.L""...","{'gene_id': 'gene48202', 'gene_name': 'mt-trna...",mt-trna-val.L,gene48202,gene48202_t
3,MT,Xenbase,exon,3093,4723,.,+,.,"gene_id ""gene44770""; gene_name ""mt-rnr2.L""; tr...","{'gene_id': 'gene44770', 'gene_name': 'mt-rnr2...",mt-rnr2.L,gene44770,gene44770_t
4,MT,Xenbase,exon,4724,4798,.,+,.,"gene_id ""gene43253""; gene_name ""mt-trna-leu1.L...","{'gene_id': 'gene43253', 'gene_name': 'mt-trna...",mt-trna-leu1.L,gene43253,gene43253_t
...,...,...,...,...,...,...,...,...,...,...,...,...,...
771990,chr9_10S,Xenbase,exon,104524309,104524351,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S,gene24857,rna64354
771992,chr9_10S,Xenbase,exon,104525747,104525868,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S,gene24857,rna64354
771994,chr9_10S,Xenbase,exon,104527008,104527068,.,-,0,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S,gene24857,rna64354
771996,chr9_10S,Xenbase,exon,104530653,104530753,.,-,2,"gene_id ""gene24857""; gene_name ""Xetrov90026942...","{'gene_id': 'gene24857', 'gene_name': 'Xetrov9...",Xetrov90026942m.S,gene24857,rna64354


In [49]:
# Extract gene_name, gene_id, and transcript_id fields to generate an ID mapping matrix (idmm)
idmm_df = models_asgtf_df[['gene_name', 'gene_id', 'transcript_id']].drop_duplicates()
idmm_df.dropna(inplace = True)
display(idmm_df)

# generate a filename and file for the idmm
idmm_filename = '_'.join([species_prefix, conditions, 'gtf-idmm.tsv'])
idmm = IdmmFile(idmm_filename, species_SampleDict, kind = 'gtf_idmm', sources = [sample_Docket.annot])

# save to file and add to the Docket
idmm_df.to_csv(idmm.path, sep = '\t')
sample_Docket.add_keyfile(idmm, 'gtf_idmm')

,gene_name,gene_id,transcript_id
0,mt-trna-phe.L,gene42065,gene42065_t
1,mt-rnr1.L,gene34778,gene34778_t
2,mt-trna-val.L,gene48202,gene48202_t
3,mt-rnr2.L,gene44770,gene44770_t
4,mt-trna-leu1.L,gene43253,gene43253_t
...,...,...,...
771955,vkorc1.S,gene13974,rna40672
771956,vkorc1.S,gene13974,rna40673
771962,LOC108703288,gene37157,rna77797
771977,LOC108702356 [provisional:urgcp],gene34762,rna74435


# 11. Generate cDNA and peptide files
Using the updated gtf file and genome file, generate cDNA sequence.

Then, using the cDNA sequence, generate peptide sequences using transdecoder.

Expect this step to take some time, probably ~20-30min.

In [50]:
cdna = sample_Docket.genome_fasta.get_transdecoder_cdna_gtf(models_asgtf_updated, TRANSDECODER_LOC)
sample_Docket.add_keyfile(cdna, 'cdna')

transdecoder_files = sample_Docket.cdna.to_pep_files(TDLONGORF_LOC, TDPREDICT_LOC)
sample_Docket.add_keyfiles(transdecoder_files)

-parsing cufflinks output: ../../output/Xlae_adultbrain/XENLA_9.2_Xenbase_updated.gtf
-parsing genome fasta: ../../output/Xlae_adultbrain/XENLA_9.2_genome.fa
-done parsing genome.
// processing MT
// processing Scaffold100
// processing Scaffold1000
// processing Scaffold100330
// processing Scaffold10049
// processing Scaffold10052
// processing Scaffold100627
// processing Scaffold10094
// processing Scaffold101
// processing Scaffold10127
// processing Scaffold10158
// processing Scaffold10166
// processing Scaffold101757
// processing Scaffold101947
// processing Scaffold101966
// processing Scaffold102
// processing Scaffold10215
// processing Scaffold10219
// processing Scaffold1022
// processing Scaffold1024
// processing Scaffold10260
// processing Scaffold10281
// processing Scaffold102870
// processing Scaffold103
// processing Scaffold1032
// processing Scaffold103277
// processing Scaffold103338
// processing Scaffold103460
// processing Scaffold10353
// processing Scaffold

CMD: touch ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir/.__checkpoints_longorfs/TD.longorfs.ok


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/exclude_similar_proteins.pl ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.top_longest_5000 > ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.top_longest_5000.nr
-skipping training candidate: rna75476.p1, not unique enough
-skipping training candidate: rna86172.p2, not unique enough
-skipping training candidate: rna74302.p1, not unique enough
-skipping training candidate: rna19792.p1, not unique enough
-skipping training candidate: rna46980.p1, not unique enough
-skipping training candidate: rna88326.p1, not unique enough
-skipping training candidate: rna36869.p1, not unique enough
-skipping training candidate: rna83367.p2, not unique enough
-skipping training candidate: rna91229.p2, not unique enough
-skipping training candidate: rna89026.p2, not unique enough
-skipping training candidate: rna37880.p1, not u

null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/compute_AUC.pl ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.feature.scores.roc


...

* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.+.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.-.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/deplete_feature_noise.pl  --features_plus ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.+.features  --pwm_minus ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.-.pwm  --out_prefix ../../output/Xlae_adultbrain/

null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/compute_AUC.pl ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.enhanced.feature.scores.roc


null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.enhanced.+.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/start_codon_refinement.pl --transcripts ../../output/Xlae_adultbrain/XENLA_9.2_genome_cDNA.fa --gff3_file ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.best_candidates.gff3 --workdir ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir/ > ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.best_candidates.gff3.revised_starts.gff3
Refining start codon selections.
-number of revised start positions: 3105
* Running CMD: cp ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_

# 12. Push files to AWS S3

Iteratively moves through the file_set and file_dict variables and populates files into the right place in AWS.

# 13. Pickle the `sample_Docket` variable for use by the next script

In [51]:
dill_filename = output_folder + '_'.join([species_prefix, conditions, 'sample_Docket.pkl'])

with open(dill_filename, 'wb') as file:
    dill.dump(sample_Docket, file)